In [1]:
# to use goolge drive as disk
from google.colab import drive 

In [3]:
# mounting gdrive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Speech-to-text_GCP
!ls

/content/gdrive/MyDrive/Colab Notebooks/Speech-to-text_GCP
api-key.json  genevieve.wav  parts  requirements.txt  STT_GCP.ipynb


## Preparing environment for this SpeechToText API

In [5]:
# Display libraries required
!cat requirements.txt 

google-api-python-client==1.12.1
httplib2==0.10.3
oauth2client==4.1.2
pyasn1==0.4.2
pyasn1-modules==0.2.1
rsa==3.4.2
six==1.15.0
SpeechRecognition==3.8.1
tqdm==4.62.2
uritemplate==3.0.0


In [6]:
# installing libraries as needed.
!pip install folium==0.2.1
!pip install -r requirements.txt
!pip install six==1.15.0

     |████████████████████████████████| 69 kB 3.1 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79809 sha256=1c3d70d8a98b2fbf5ddd26338c697d742b80981fbf67dc8616e7372217e972d6
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 61 kB 3.0 MB/s 
     |████████████████████████████████| 204 kB 8.6 MB/s 
     |████████████████████████████████| 99 kB 8.4 MB/s 
     |████████████████████████████████| 71 kB 9.1 MB/s 
     |████████████████████████████████| 60 kB 8.1 MB/s 
     |████████████████████████████████| 46 kB 4.4 MB/s 
     |████████████████████████████████| 32.8 MB 36 kB/s 
  Created wheel for httplib2: filename=httplib2-0.10.3-py3-none-any.whl size=83987 sha256=09205bbce036fe86db8ec59f1

In [7]:
# Break the audio into 30 sec parts
!ffmpeg -i genevieve.wav -f segment -segment_time 30 -c copy parts/out%09d.wav

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [8]:
# from SpeechRecognition package, import speech_recognition module
import speech_recognition as sr

import os
from tqdm import tqdm

# For multi processing
from multiprocessing.dummy import Pool
# Number of concurrent threads
pool = Pool(8) 

from IPython.display import Audio

In [9]:
# Read the contents of the API File
with open("api-key.json") as f:
    GOOGLE_CLOUD_SPEECH_CREDENTIALS = f.read()

In [10]:
# get the recogniser from the speech_recognition api
r = sr.Recognizer()

### Speech recognition engine/API support:

*   CMU Sphinx (works offline)
*   List item
*   Google Speech Recognition
*    Google Cloud Speech API
*    Wit.ai
*    Microsoft Azure Speech
*    Microsoft Bing Voice Recognition (Deprecated)
*    Houndify API
*    IBM Speech to Text
*    Snowboy Hotword Detection (works offline)


In [11]:
# get the list of all the files that we want to convert
files = sorted(os.listdir('parts/'))
files

['out000000000.wav', 'out000000001.wav', 'out000000002.wav']

In [12]:
name = 'parts/'+files[0]
Audio(name)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
# Load audio file
with sr.AudioFile(name) as source:
    audio = r.record(source)
audio

In [14]:
# Transcribe audio file
text = r.recognize_google_cloud(
    audio,
    credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS
    )
text

'this Dynamic Workshop aims to provide up-to-date information on pharmacological approaches, issues, and treatment in the geriatric population to assist in preventing medication related problems, appropriately and effectively managing medications and compliance. The concept of polypharmacy parentheses taking multiple types of drugs parentheses will also be discussed, as though '

In [15]:
def transcribe(data):
    idx, file = data
    name = "parts/" + file
    print(name + " started")
    # Load audio file
    with sr.AudioFile(name) as source:
        audio = r.record(source)
    # Transcribe audio file
    text = r.recognize_google_cloud(audio, credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS)
    print(name + " done")
    return {
        "idx": idx,
        "text": text
    }

In [16]:
# running them in parallel to make it faster
all_text = pool.map(transcribe, enumerate(files))
pool.close()
pool.join()

parts/out000000000.wav started
parts/out000000001.wav started
parts/out000000002.wav started
parts/out000000002.wav done
parts/out000000001.wav done
parts/out000000000.wav done


In [17]:
all_text

[{'idx': 0,
  'text': 'this Dynamic Workshop aims to provide up-to-date information on pharmacological approaches, issues, and treatment in the geriatric population to assist in preventing medication related problems, appropriately and effectively managing medications and compliance. The concept of polypharmacy parentheses taking multiple types of drugs parentheses will also be discussed, as though '},
 {'idx': 1,
  'text': 'is a common issue that can impact adverse side effects in the geriatric population. Participants will leave with the knowledge and considerations of common drug interactions and how to minimize the effects that limit function. Summit professional education is approved provider of continuing education. This course is offered for 6. '},
 {'idx': 2,
  'text': '. discourse contains a Content classified under the both the domain of occupational therapy and professional issues. '}]

In [ ]:
from __future__ import division

import re
import sys

from google.cloud import speech

import pyaudio
from six.moves import queue

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms


class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""

    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b"".join(data)


def listen_print_loop(responses):
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    """
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        # The `results` list is consecutive. For streaming, we only care about
        # the first result being considered, since once it's `is_final`, it
        # moves on to considering the next utterance.
        result = response.results[0]
        if not result.alternatives:
            continue

        # Display the transcription of the top alternative.
        transcript = result.alternatives[0].transcript

        # Display interim results, but with a carriage return at the end of the
        # line, so subsequent lines will overwrite them.
        #
        # If the previous result was longer than this one, we need to print
        # some extra spaces to overwrite the previous result
        overwrite_chars = " " * (num_chars_printed - len(transcript))

        if not result.is_final:
            sys.stdout.write(transcript + overwrite_chars + "\r")
            sys.stdout.flush()

            num_chars_printed = len(transcript)

        else:
            print(transcript + overwrite_chars)

            # Exit recognition if any of the transcribed phrases could be
            # one of our keywords.
            if re.search(r"\b(exit|quit)\b", transcript, re.I):
                print("Exiting..")
                break

            num_chars_printed = 0


def main():
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = "en-US"  # a BCP-47 language tag

    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code,
    )

    streaming_config = speech.StreamingRecognitionConfig(
        config=config, interim_results=True
    )

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (
            speech.StreamingRecognizeRequest(audio_content=content)
            for content in audio_generator
        )

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_loop(responses)


if __name__ == "__main__":
    main()

ImportError: ignored